# Sample Workflow for d3rlpy Experiments

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [74]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_stochpid.txt")
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [75]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02 -3.69999953e-02
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.50429671e-01 -4.92727243e-01 -5.31666025e-03]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.25610892e-01 -3.35999953e-02
 -2.42000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08749986e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90489108e-01 -5.87999953e-02
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.76979602e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48010892e-01  4.64000047e-02
 -1.04000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.83151637e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.08389108e-01  3.32000047e-02
 -2.02000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.25137655

[ 0.00000000e+00  7.95731469e+08 -1.06389108e-01 -5.79999531e-03
 -2.02000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -3.87850514e-02  6.00000000e-01]
Read chunk # 54 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.79891077e-02 -1.81999953e-02
  2.77999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -5.85850097e-01  6.00000000e-01]
Read chunk # 55 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.85310892e-01  5.70000047e-02
  2.41999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.22471359e-01  4.41606904e-02 -6.00000000e-01]
Read chunk # 56 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.14889108e-01  1.36000047e-02
 -4.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.81214600e-01 -5.19425255e-01  6.00000000e-01]
Read chunk # 57 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.65110892e-01 -3.07999953e-02
  2.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.65982652e-

start
[ 0.00000000e+00  7.95731469e+08 -3.52891077e-02  3.98000047e-02
  1.18999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.70574617e-01 -4.20063674e-01  6.00000000e-01]
Read chunk # 113 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.24289108e-01  5.88000047e-02
 -3.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -8.79577116e-02 -3.28703820e-01  3.22520023e-01]
Read chunk # 114 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.93189108e-01  5.68000047e-02
  1.21999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.05551438e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 115 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.05910892e-01 -3.91999953e-02
 -1.24000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.38197550e-01  4.97490391e-01 -6.00000000e-01]
Read chunk # 116 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.63889108e-01 -2.27999953e-02
 -1.57000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.

start
[ 0.00000000e+00  7.95731469e+08 -3.47889108e-01 -4.59999531e-03
  2.85999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  1.59959033e-01  6.00000000e-01]
Read chunk # 164 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  5.29108923e-02 -1.79999953e-02
 -6.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.55179412e-01 -4.05280444e-02 -6.00000000e-01]
Read chunk # 165 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.34689108e-01  2.48000047e-02
 -2.20000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.97320588e-01 -3.53911963e-01  6.00000000e-01]
Read chunk # 166 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.16589108e-01  4.18000047e-02
 -5.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.07637733e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 167 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.53910892e-01 -5.39999531e-03
 -1.10000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.

start
[ 0.00000000e+00  7.95731469e+08 -4.40589108e-01 -1.39999531e-03
 -3.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.69679248e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 226 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.15989108e-01  5.66000047e-02
 -1.41000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.52867603e-01  6.00000000e-01]
Read chunk # 227 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.99010892e-01 -1.35999953e-02
  1.85999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.67610465e-01  2.95995569e-01 -6.00000000e-01]
Read chunk # 228 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.80689108e-01 -3.11999953e-02
 -2.41000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.45059320e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 229 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.36289108e-01 -2.53999953e-02
  1.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.

...

start
[ 0.00000000e+00  7.95731469e+08  1.15310892e-01 -3.09999953e-02
  1.07999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  2.08428605e-01 -6.00000000e-01]
Read chunk # 1761 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.51589108e-01  5.80000047e-02
 -2.45000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  3.09460000e-02  6.00000000e-01]
Read chunk # 1762 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.40789108e-01 -4.19999531e-03
  2.65999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.86347535e-01 -4.76395769e-01  6.00000000e-01]
Read chunk # 1763 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.31110892e-01 -1.35999953e-02
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.46025294e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1764 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.13289108e-01  5.70000047e-02
 -7.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


[ 0.00000000e+00  7.95731469e+08 -1.13289108e-01  3.20000469e-03
 -4.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.51715294e-01 -3.07583996e-01  6.00000000e-01]
Read chunk # 1815 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.81510892e-01  5.36000047e-02
 -1.88000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1816 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.21891077e-02  3.50000047e-02
 -1.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.10099086e-01 -9.38917286e-02  5.71962204e-02]
Read chunk # 1817 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.57210892e-01  2.58000047e-02
 -1.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.74173370e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1818 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.83089108e-01  1.96000047e-02
 -4.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.12

start
[ 0.00000000e+00  7.95731469e+08 -1.58589108e-01 -3.79999953e-02
 -2.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.09434744e-01 -5.42482642e-01  6.00000000e-01]
Read chunk # 1872 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.46710892e-01 -1.59999531e-03
 -2.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.67998584e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1873 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.32110892e-01 -4.43999953e-02
  1.18999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.31428997e-01  4.88928876e-01 -6.00000000e-01]
Read chunk # 1874 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.81689108e-01  8.80000469e-03
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.41285162e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1875 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.75108923e-02  2.38000047e-02
  4.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


start
[ 0.00000000e+00  7.95731469e+08  1.42610892e-01  4.96000047e-02
 -2.22000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1926 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.99889108e-01 -1.91999953e-02
 -2.74000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.67670043e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1927 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.54108923e-02  9.00000469e-03
  2.03999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  1.49009041e-01 -6.00000000e-01]
Read chunk # 1928 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.75210892e-01  2.68000047e-02
  4.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.27665800e-02  3.82498050e-01 -6.00000000e-01]
Read chunk # 1929 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.34210892e-01  1.74000047e-02
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


start
[ 0.00000000e+00  7.95731469e+08 -4.27989108e-01  1.02000047e-02
  1.41999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.92519189e-01 -5.23299995e-01  6.00000000e-01]
Read chunk # 1976 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.93889108e-01  1.10000047e-02
  3.69998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  8.71793504e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 1977 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.18210892e-01 -3.33999953e-02
 -3.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  8.20425378e-02  5.61438197e-01 -6.00000000e-01]
Read chunk # 1978 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.00910892e-01 -5.91999953e-02
 -1.74000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.93353866e-01  1.65413557e-01 -6.00000000e-01]
Read chunk # 1979 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.24389108e-01  1.74000047e-02
 -1.10000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


In [76]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -127.88257,
 'std': 81.13078,
 'min': -410.7058,
 'max': 0.0,
 'histogram': (array([  6,   7,  19,  21,  33,  35,  51,  58,  59,  50,  70,  93, 151,
         178, 209, 220, 273, 353, 112,   1]),
  array([-410.7058  , -390.17053 , -369.63522 , -349.09995 , -328.56464 ,
         -308.02936 , -287.49408 , -266.95877 , -246.4235  , -225.8882  ,
         -205.3529  , -184.81761 , -164.28232 , -143.74704 , -123.21175 ,
         -102.67645 ,  -82.14116 ,  -61.605873,  -41.07058 ,  -20.53529 ,
            0.      ], dtype=float32))}

In [77]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [78]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [79]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

-0.09837139175653559


In [80]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [81]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=2, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-17 23:17.39 [debug    ] RoundIterator is selected.
2022-04-17 23:17.39 [info     ] Directory is created at d3rlpy_logs/CQL_20220417231739
2022-04-17 23:17.39 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-17 23:17.39 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-17 23:17.39 [warning  ] Skip building models since they're already built.
2022-04-17 23:17.39 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220417231739/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/2:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 23:30.32 [info     ] CQL_20220417231739: epoch=1 step=6928 epoch=1 metrics={'time_sample_batch': 0.0003384330737948693, 'time_algorithm_update': 0.1091889642678012, 'temp_loss': 2.7889417736139364, 'temp': 0.7468700823535407, 'alpha_loss': -3.8629026590401114, 'alpha': 1.2555583591205144, 'critic_loss': 25.875352859910034, 'actor_loss': 26.26949927444741, 'time_step': 0.10996188202713708, 'td_error': 41.15516291492437, 'init_value': -67.10748291015625, 'ave_value': -63.22335990977519} step=6928
2022-04-17 23:30.32 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417231739/model_6928.pt


Epoch 2/2:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 23:43.26 [info     ] CQL_20220417231739: epoch=2 step=13856 epoch=2 metrics={'time_sample_batch': 0.00034043323635779684, 'time_algorithm_update': 0.10931617620360218, 'temp_loss': 0.8610301685259263, 'temp': 0.4212918789459362, 'alpha_loss': 0.6399822184131179, 'alpha': 1.1888980756386873, 'critic_loss': 111.8939822971752, 'actor_loss': 88.44204160963416, 'time_step': 0.11011452690696055, 'td_error': 83.29594987256363, 'init_value': -123.05992126464844, 'ave_value': -116.16129022357225} step=13856
2022-04-17 23:43.26 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417231739/model_13856.pt


[(1,
  {'time_sample_batch': 0.0003384330737948693,
   'time_algorithm_update': 0.1091889642678012,
   'temp_loss': 2.7889417736139364,
   'temp': 0.7468700823535407,
   'alpha_loss': -3.8629026590401114,
   'alpha': 1.2555583591205144,
   'critic_loss': 25.875352859910034,
   'actor_loss': 26.26949927444741,
   'time_step': 0.10996188202713708,
   'td_error': 41.15516291492437,
   'init_value': -67.10748291015625,
   'ave_value': -63.22335990977519}),
 (2,
  {'time_sample_batch': 0.00034043323635779684,
   'time_algorithm_update': 0.10931617620360218,
   'temp_loss': 0.8610301685259263,
   'temp': 0.4212918789459362,
   'alpha_loss': 0.6399822184131179,
   'alpha': 1.1888980756386873,
   'critic_loss': 111.8939822971752,
   'actor_loss': 88.44204160963416,
   'time_step': 0.11011452690696055,
   'td_error': 83.29594987256363,
   'init_value': -123.05992126464844,
   'ave_value': -116.16129022357225})]

In [82]:
model.save_model('cqlStochpid2000_Ep40_CPUonly.pt')
model.save_policy('cqlStochpid2000_Ep40_CPUonly.pt')

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [83]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [84]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [85]:
from d3rlpy.torch_utility import to_cpu
to_cpu(model)
model.save_policy("cqlStochpid2000Ep40CPU.pt")
model.save_model("cqlStochpid2000Ep40modelCPU.pt")

In [86]:
# for key in dir(model):
#     module = getattr(model, key)
#     if isinstance(module, (torch.nn.Module, torch.nn.Parameter)):
#         print(yes)
#         print(key)
# dir(model)
# type(model)
# model.cpu()
# from d3rlpy.algos.torch.base import TorchImplBase
# new_model = TorchImplBase()
# from d3rlpy.torch_utility import _get_attributes
# model._device = "cpu:0"
# print(model._device)


# def my_get_state_dict(impl: Any) -> Dict[str, Any]:
#     rets = {}
#     for key in _get_attributes(impl):
#         obj = getattr(impl, key)
#         if isinstance(obj, (torch.nn.Module, torch.optim.Optimizer)):
#             if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#                 obj.cpu()
#             rets[key] = obj.state_dict()
#     return rets

# torch.save(my_get_state_dict(model), "my_test_model.pt")

# for key in dir(model):
#     obj = getattr(model, key)
#     if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#         obj.cpu()
#         print("convert to cpu")
# model.save_policy("cqlStochpid2000Ep40modelCPU.pt")

# import trace
# tracer = trace.Trace()
# tracer.run('model.save_policy("cqlStochpid2000Ep40modelCPU.pt")')